# 1 - Data Access and Download

In [1]:
import os, sys, re, ast
import pandas as pd
import numpy as np
import xarray as xr

from ooi_data_explorations.common import load_kdata

from ooinet import M2M

### Available Gliders and Datasets

##### ADCP Datasets

In [2]:
adcp_datasets = M2M.search_datasets(array='CP05MOAS', instrument='ADCP', English_names=True)
adcp_datasets

Searching https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS


,array,array_name,node,node_name,instrument,instrument_name,refdes,url,deployments
0,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL559,Coastal Glider 559,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL559-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
1,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL514,Coastal Glider 514,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL514-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3]"
2,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL389,Coastal Glider 389,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL389-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6]"
3,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL388,Coastal Glider 388,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL388-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
4,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL387,Coastal Glider 387,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL387-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8]"
5,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL380,Coastal Glider 380,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL380-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
6,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL379,Coastal Glider 379,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL379-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
7,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL376,Coastal Glider 376,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL376-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
8,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL374,Coastal Glider 374,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL374-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
9,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL340,Coastal Glider 340,01-ADCPAM000,Velocity Profiler (600kHz),CP05MOAS-GL340-01-ADCPAM000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


##### Engineering Datasets

In [3]:
eng_datasets = M2M.search_datasets(array='CP05MOAS', instrument='ENG', English_names=True)
eng_datasets

Searching https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS


,array,array_name,node,node_name,instrument,instrument_name,refdes,url,deployments
0,CP05MOAS,Coastal Pioneer NES Mobile Assets,PG583,Profiling Glider 583,00-ENG000000,Mobile Asset Controller,CP05MOAS-PG583-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4]"
1,CP05MOAS,Coastal Pioneer NES Mobile Assets,PG564,Profiling Glider 564,00-ENG000000,Mobile Asset Controller,CP05MOAS-PG564-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
2,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL559,Coastal Glider 559,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL559-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5]"
3,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL514,Coastal Glider 514,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL514-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3]"
4,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL389,Coastal Glider 389,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL389-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6]"
5,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL388,Coastal Glider 388,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL388-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
6,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL387,Coastal Glider 387,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL387-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8]"
7,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL380,Coastal Glider 380,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL380-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
8,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL379,Coastal Glider 379,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL379-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9,CP05MOAS,Coastal Pioneer NES Mobile Assets,GL376,Coastal Glider 376,00-ENG000000,Mobile Asset Controller,CP05MOAS-GL376-00-ENG000000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


##### Select an ADCP and associated Engineering dataset

In [4]:
# For this example we will be using CP05MOAS-GL379 D00008, which was deployed at the Pioneer NES array from 2019-09-27 to 2019-11-19.
refdes = 'CP05MOAS-GL379-01-ADCPAM000'
eng_refdes = 'CP05MOAS-GL379-00-ENG000000'
deployment = 8

In [5]:
dep_str = f"{deployment:04d}"  # always 4 digits, e.g., '0008'
glider_name = '-'.join(refdes.split('-')[:2]) # shorten ref des to get glider name

### Download ADCP Data

##### Check available datastreams

In [6]:
M2M.get_datastreams(refdes)

,refdes,method,stream
0,CP05MOAS-GL379-01-ADCPAM000,recovered_host,adcp_bottom_track_config
1,CP05MOAS-GL379-01-ADCPAM000,recovered_host,adcp_bottom_track_earth
2,CP05MOAS-GL379-01-ADCPAM000,recovered_host,adcp_config
3,CP05MOAS-GL379-01-ADCPAM000,recovered_host,adcp_engineering
4,CP05MOAS-GL379-01-ADCPAM000,recovered_host,adcp_velocity_earth


In [7]:
M2M.get_deployments(refdes)

,deploymentNumber,uid,assetId,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,1,CGINS-ADCPAM-50865,2091,39.96155,-70.90805,1000.0,2014-04-15 13:15:00,2014-07-11 12:32:00,KN217,CT20140711
1,2,CGINS-ADCPAM-50865,2091,39.94048,-70.82862,1000.0,2015-05-09 12:31:00,2015-07-02 12:26:00,AT27,MH20150702
2,3,CGINS-ADCPAM-50865,2091,39.99916,-70.70673,1000.0,2016-01-21 15:00:00,2016-05-18 02:56:00,SI20160122,AR4
3,4,CGINS-ADCPAM-50865,2091,39.917183,-70.40035,1000.0,2017-01-16 12:46:00,2017-01-17 07:00:00,WJ20170115,WJ20170115
4,5,CGINS-ADCPAM-50865,2091,39.78033,-70.47605,1000.0,2017-04-19 20:27:23,2017-04-20 11:39:41,SI20170418,SI20170418
5,6,CGINS-ADCPAM-50865,2091,39.9161,-70.36985,1000.0,2019-02-05 07:54:00,2019-05-16 22:54:03,TS20190205,TS20190516
6,7,CGINS-ADCPAM-50865,2091,39.9156,-70.9142,1000.0,2019-06-18 23:14:29,2019-07-28 02:39:00,WJ20190617,MZ20190727
7,8,CGINS-ADCPAM-50865,2091,40.0498,-70.6001,1000.0,2019-09-27 12:22:00,2019-11-19 04:35:00,AR39,AR41
8,9,CGINS-ADCPAM-50865,2091,39.9163,-70.36053,1000.0,2020-03-02 09:44:16,2020-05-22 17:09:47,TS20200301,TS20200522
9,10,CGINS-ADCPAM-50865,2091,39.88515,-70.39692,1000.0,2020-08-19 17:18:00,2020-11-04 20:26:00,TS20200819,AR48


In [8]:
site, node, sensor = refdes.split("-", 2)
method = 'recovered_host'

# Load the velocity data
stream = 'adcp_velocity_earth'
adcp_data = load_kdata(site, node, sensor, method, stream, tag=f'deployment{dep_str}*{refdes}*.nc')

# Load the engineering data
stream = 'adcp_engineering'
adcp_engineering = load_kdata(site, node, sensor, method, stream, tag=f'deployment{dep_str}*{refdes}*.nc')

# Load the config data
stream = 'adcp_config'
adcp_config = load_kdata(site, node, sensor, method, stream, tag=f'deployment{dep_str}*{refdes}*.nc')

# Load the bottom track data
stream = 'adcp_bottom_track_earth'
adcp_bottom_track = load_kdata(site, node, sensor, method, stream, tag=f'deployment{dep_str}*{refdes}*.nc')


Loading and Processing Data Files: 100%|██████████| 2/2 [00:27<00:00, 13.50s/it]


Merging the data files into a single dataset


Loading and Processing Data Files: 100%|██████████| 1/1 [00:14<00:00, 14.85s/it]


Merging the data files into a single dataset


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  7.22it/s]


Merging the data files into a single dataset


Loading and Processing Data Files: 100%|██████████| 1/1 [00:18<00:00, 18.41s/it]

Merging the data files into a single dataset


###### Save the ADCP Data & associated datasets

In [9]:
# ADCP Data
adcp_data.to_netcdf(f"../data/raw/{refdes}.deployment{dep_str}.adcp_data.nc", format="netcdf4", engine="h5netcdf")

# ADCP Engineering
adcp_engineering.to_netcdf(f"../data/raw/{refdes}.deployment{dep_str}.adcp_engineering.nc", format="netcdf4", engine="h5netcdf")

# ADCP Config
adcp_config.to_netcdf(f"../data/raw/{refdes}.deployment{dep_str}.adcp_config.nc", format="netcdf4", engine="h5netcdf")

# ADCP Bottom Track
adcp_bottom_track.to_netcdf(f"../data/raw/{refdes}.deployment{dep_str}.adcp_bottom_track.nc", format="netcdf4", engine="h5netcdf")

##### Glider Data
Next, need to get the glider engineering data which has the key data we will need for calculating differences between waypoints and surfacings. This is not available via netCDF file or on kdata, so will need to make a THREDDS request and download the separate csv files

In [10]:
M2M.get_datastreams(eng_refdes)

,refdes,method,stream
0,CP05MOAS-GL379-00-ENG000000,recovered_host,glider_eng_metadata_recovered
1,CP05MOAS-GL379-00-ENG000000,recovered_host,glider_eng_recovered
2,CP05MOAS-GL379-00-ENG000000,recovered_host,glider_eng_sci_recovered
3,CP05MOAS-GL379-00-ENG000000,recovered_host,glider_gps_position
4,CP05MOAS-GL379-00-ENG000000,telemetered,glider_eng_metadata
5,CP05MOAS-GL379-00-ENG000000,telemetered,glider_eng_sci_telemetered
6,CP05MOAS-GL379-00-ENG000000,telemetered,glider_eng_telemetered
7,CP05MOAS-GL379-00-ENG000000,telemetered,glider_gps_position


In [11]:
M2M.get_deployments(refdes, deploy_num=dep_str)

,deploymentNumber,uid,assetId,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,8,CGINS-ADCPAM-50865,2091,40.0498,-70.6001,1000.0,2019-09-27 12:22:00,2019-11-19 04:35:00,AR39,AR41


In [12]:
# Extract the start and end times for the deployment 
deploy_info = M2M.get_deployments(refdes, deploy_num=dep_str)
deploy_start = deploy_info['deployStart'].iloc[0].strftime("%Y-%m-%d")
deploy_end   = deploy_info['deployEnd'].iloc[0].strftime("%Y-%m-%d")

In [13]:
from ooinet.M2M import SESSION
from bs4 import BeautifulSoup

# Request the dataset
# Uncomment the following line to make the initial request and then comment it out again
#thredds_url = M2M.get_thredds_url(eng_refdes, 'recovered_host', 'glider_eng_recovered', goldCopy=False, beginDT=deploy_start, endDT=deploy_end, format="application/csv")
# Uncomment the following line after you have your direct download THREDDS link from your email so you only have to make the request once
thredds_url = 'https://downloads.oceanobservatories.org/async_results/cdobson@whoi.edu/20250926T165905868Z-CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered/' # THREDDS request are good for ~6 months

# Access the THREDDS catalog of the available datasets
page = SESSION.get(thredds_url).text
soup = BeautifulSoup(page, "html.parser")
pattern = re.compile('.*\\.csv$')
catalog = sorted([node.get('href') for node in soup.find_all('a', text=pattern)])
catalog

/tmp/ipykernel_2069/988027866.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  catalog = sorted([node.get('href') for node in soup.find_all('a', text=pattern)])


['deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20190927T122653-20190927T235959.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20190928T000003-20190928T235958.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20190929T000003-20190929T235959.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20190930T000004-20190930T235959.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20191001T000024-20191001T235955.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20191002T000000-20191002T235938.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20191003T000005-20191003T235956.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-recovered_host-glider_eng_recovered_20191004T000000-20191004T235956.csv',
 'deployment0008_CP05MOAS-GL379-00-ENG000000-rec

In [15]:
# load the glider engineering data
glider_engineering = pd.DataFrame()
for file in catalog:
    df = pd.read_csv(thredds_url + file)
    glider_engineering = pd.concat([glider_engineering, df], ignore_index=True)

glider_engineering.head()

,obs,c_air_pump,c_ballast_pumped,c_battpos,c_battroll,c_bsipar_on,c_de_oil_vol,c_dvl_on,c_flbbcd_on,c_heading,...,m_water_vy,m_why_started,m_x_lmc,m_y_lmc,port_timestamp,preferred_timestamp,time,x_last_wpt_lat,x_last_wpt_lon,x_system_clock_adjusted
0,"('0',)",1,260.0,0.7,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,16,-10.2362,-41.8981,0.0,internal_timestamp,3.778576e+09,40.083332,-70.375,0.0
1,"('1',)",1,260.0,0.7,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,16,0.0000,0.0000,0.0,internal_timestamp,3.778576e+09,40.083332,-70.375,0.0
2,"('2',)",1,NaN,0.7,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,-99,NaN,NaN,0.0,internal_timestamp,3.778576e+09,NaN,NaN,NaN
3,"('3',)",1,NaN,0.7,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,-99,NaN,NaN,0.0,internal_timestamp,3.778576e+09,NaN,NaN,NaN
4,"('4',)",1,NaN,0.7,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,-99,NaN,NaN,0.0,internal_timestamp,3.778576e+09,NaN,NaN,NaN


In [16]:
from ooinet.utils import ntp_seconds_to_datetime

glider_engineering['time'] = glider_engineering['time'].apply(lambda x: ntp_seconds_to_datetime(x))
glider_engineering = glider_engineering.set_index(keys='time')
glider_engineering.head()

,obs,c_air_pump,c_ballast_pumped,c_battpos,c_battroll,c_bsipar_on,c_de_oil_vol,c_dvl_on,c_flbbcd_on,c_heading,...,m_water_vx,m_water_vy,m_why_started,m_x_lmc,m_y_lmc,port_timestamp,preferred_timestamp,x_last_wpt_lat,x_last_wpt_lon,x_system_clock_adjusted
time,,,,,,,,,,,,,,,,,,,,,
2019-09-27 12:26:53.537450,"('0',)",1,260.0,0.7,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,0.0,16,-10.2362,-41.8981,0.0,internal_timestamp,40.083332,-70.375,0.0
2019-09-27 12:27:35.679110,"('1',)",1,260.0,0.7,0.0,-1.0,260.0,-1.0,-1.0,0.0,...,0.0,0.0,16,0.0000,0.0000,0.0,internal_timestamp,40.083332,-70.375,0.0
2019-09-27 12:27:49.803800,"('2',)",1,NaN,0.7,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,NaN,-99,NaN,NaN,0.0,internal_timestamp,NaN,NaN,NaN
2019-09-27 12:27:54.485140,"('3',)",1,NaN,0.7,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,NaN,-99,NaN,NaN,0.0,internal_timestamp,NaN,NaN,NaN
2019-09-27 12:27:59.148500,"('4',)",1,NaN,0.7,0.0,-1.0,260.0,-1.0,-1.0,NaN,...,NaN,NaN,-99,NaN,NaN,0.0,internal_timestamp,NaN,NaN,NaN


###### Save the glider engineering dataset

In [17]:
# Save the dataset
glider_engineering.to_csv(f"../data/raw/deployment{dep_str}_{eng_refdes}-recovered_host-glider_eng_recovered.csv")

---
## Science Data
Next, download the associated glider data from the IOOS GDAC

In [18]:
from erddapy import ERDDAP

In [19]:
# Connect to the IOOS GDAC ERDDAP server
e = ERDDAP(server='https://gliders.ioos.us/erddap')

# Grab the search url and look for OOI gliders
url = e.get_search_url(search_for="ooi", response="csv")

# Get available OOI dataset
df = pd.read_csv(url)

# List the datasets with the glider that we want
glider_id = 'cp_379'
[x for x in df['Dataset ID'] if glider_id in x]

['cp_379-20240404T2244-delayed',
 'cp_379-20240523T1332-delayed',
 'cp_379-20240630T1308-delayed',
 'cp_379-20241120T1605-delayed',
 'cp_379-20240523T1332',
 'cp_379-20240630T1308',
 'cp_379-20241120T1605',
 'cp_379-20140415T1315-delayed',
 'cp_379-20150509T1231-delayed',
 'cp_379-20160121T1500-delayed',
 'cp_379-20190205T0754-delayed',
 'cp_379-20190618T2314-delayed',
 'cp_379-20190927T1222-delayed',
 'cp_379-20200302T0944-delayed',
 'cp_379-20200819T1718-delayed',
 'cp_379-20210131T1757-delayed',
 'cp_379-20210517T2048-delayed',
 'cp_379-20210903T2251-delayed',
 'cp_379-20220104T2052-delayed',
 'cp_379-20220518T2012-delayed',
 'cp_379-20200302T0944',
 'cp_379-20200819T1718',
 'cp_379-20210517T2048',
 'cp_379-20210903T2251',
 'cp_379-20220104T2052',
 'cp_379-20220518T2012']

In [20]:
# Set the dataset constraints
e.dataset_id = 'cp_379-20190927T1222-delayed'
e.protocol = 'tabledap'
e.constraints = None
data_url = e.get_download_url(response='opendap')

In [22]:
# Download the data
gdac = e.to_xarray()
gdac['time'] = gdac['precise_time']
gdac = gdac.drop_vars(names='precise_time')
gdac = gdac.swap_dims({'obs':'time'})
gdac.load()

<xarray.Dataset> Size: 1GB
Dimensions:                                  (trajectory: 1, profile: 2770,
                                              time: 3805170)
Coordinates:
  * trajectory                               (trajectory) object 8B 'cp_379-2...
  * time                                     (time) datetime64[ns] 30MB 2019-...
    latitude                                 (profile) float64 22kB 40.05 ......
    longitude                                (profile) float64 22kB -70.6 ......
    lat_uv                                   (profile) float64 22kB 40.05 ......
    lon_uv                                   (profile) float64 22kB -70.6 ......
    time_uv                                  (profile) datetime64[ns] 22kB 20...
    depth                                    (time) float32 15MB nan ... 3.731
Dimensions without coordinates: profile
Data variables: (12/56)
    wmo_id                                   (trajectory) object 8B '4801971'
    profile_id                               (profile) float64 22kB 1.0 ... 2...
    u                                        (profile) float64 22kB 0.04186 ....
    v                                        (profile) float64 22kB -0.03903 ...
    trajectoryIndex                          (profile) int32 11kB 0 0 0 ... 0 0
    rowSize                                  (profile) int32 11kB 564 ... 575
    ...                                       ...
    qartod_temperature_spike_flag            (time) float32 15MB nan nan ... 1.0
    radiation_wavelength                     (time) float64 30MB 700.0 ... 700.0
    roll                                     (time) float64 30MB -1.1 ... 1.4
    salinity                                 (time) float32 15MB nan ... 34.35
    source_file                              (time) object 30MB 'cp_379-2019-...
    temperature                              (time) float32 15MB nan ... 15.87
Attributes: (12/73)
    acknowledgment:                 Funding provided by the National Science ...
    cdm_data_type:                  TrajectoryProfile
    cdm_profile_variables:          time_uv,lat_uv,lon_uv,u,v,profile_id,time...
    cdm_trajectory_variables:       trajectory,wmo_id
    comment:                        Deployment summary: This glider was deplo...
    contributor_name:               Al Plueddemann, Derek Buffitt, Peter Bric...
    ...                             ...
    time_coverage_end:              2019-11-18T20:25:38Z
    time_coverage_resolution:       PT01.018264S
    time_coverage_start:            2019-09-27T12:33:48Z
    title:                          cp_379-20190927T1222-delayed
    uuid:                           0afb5d2e-6771-424d-8e6d-5f36f6fa1c4e
    Westernmost_Easting:            -71.18003982153505

In [25]:
# Save the GDAC dataset
gdac.to_netcdf(f"../data/raw/{glider_name}.deployment{dep_str}.science.nc", format='netcdf4', engine='h5netcdf')

---
# Other Metadata

This includes getting the mooring locations and other metdata

##### Mooring location data

In [27]:
github_url = "https://raw.githubusercontent.com/oceanobservatories/asset-management/refs/heads/master/deployment"

mooring_locations = {
    'mooring':[],
    'lat': [],
    'lon': [],
    'depth': []
}

for mooring in M2M.get_api(M2M.URLS['vocab']):
    if mooring.startswith(('CP','GA','GI','GS','GP')) and 'MOAS' not in mooring:
        # Get the data
        df = pd.read_csv(f'{github_url}/{mooring}_Deploy.csv')
        lat = df['lat'].mean()
        lon = df['lon'].mean()
        depth = df['water_depth'].mean()
        # Save the data
        mooring_locations['mooring'].append(mooring)
        mooring_locations['lat'].append(lat)
        mooring_locations['lon'].append(lon)
        mooring_locations['depth'].append(depth)

mooring_locations = pd.DataFrame(mooring_locations)
mooring_locations

,mooring,lat,lon,depth
0,CP01CNPM,40.136172,-70.774114,133.000000
1,CP01CNSM,40.136497,-70.774589,134.031250
2,CP01CNSP,40.137813,-70.777408,133.250000
3,CP02PMCI,40.226715,-70.884176,126.875000
4,CP02PMCO,40.098392,-70.883301,147.411765
5,CP02PMUI,40.364950,-70.775333,94.411765
6,CP02PMUO,39.941455,-70.775943,448.842105
7,CP03ISPM,40.363231,-70.882505,92.000000
8,CP03ISSM,40.364041,-70.883005,93.530648
9,CP03ISSP,40.365247,-70.886730,94.500000


In [28]:
mooring_locations.to_csv('../data/external/mooring_locations.csv', index=False)